In [14]:
import os
import pathlib

import dask.distributed
import dotenv
import numpy as np
import xarray as xr
import xroms

import sb_basin

In [16]:
def setup_local_cluster(
    ip_address: str | None = None, 
    port: int = 8889, 
    local_directory: os.PathLike | None = None
) -> tuple[
        dask.distributed.LocalCluster, 
        dask.distributed.Client
    ]:
    """Setup a local cluster and dsak client."""
    if ip_address is None:
        ip_address = os.environ["SLURMD_NODENAME"]
    dashboard_address = f"{ip_address}:{port}"
    if local_directory is None:
        local_directory = os.environ["L_SCRATCH"]
    cluster = dask.distributed.LocalCluster(dashboard_address=dashboard_address, local_directory=local_directory)
    client = dask.distributed.Client(cluster)
    return cluster, client

cluster, client = setup_local_cluster(port=8000)

/home/groups/leift/hilditch/projects/roms_analysis_demo/project_code/.venv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8000 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41692 instead
  warnings.warn(


In [17]:
# get paths
ENV_VARIABLES = dotenv.dotenv_values(interpolate=True)
MODEL_OUTPUT_DIR = ENV_VARIABLES['SANTA_BARBARA_BASIN_MODEL_OUTPUT']
AVG_FILES = sb_basin.get_avg_files(MODEL_OUTPUT_DIR, glob="*_avg.*.nc")
GRD_FILE = sb_basin.get_grid_file(MODEL_OUTPUT_DIR, glob="*_grd.nc")

In [18]:
ds = sb_basin.open_dataset(AVG_FILES[1:3], GRD_FILE)
ds["Vtransform"] = 2
ds = ds.chunk(eta_rho=-1, xi_rho=-1, xi_u=-1, eta_v=-1, s_rho=-1, s_w=-1)
ds, xgrid = xroms.roms_dataset(ds)
ds

<xarray.Dataset> Size: 535GB
Dimensions:     (ocean_time: 48, eta_rho: 852, xi_rho: 1702, xi_u: 1701,
                 eta_v: 851, s_rho: 50, s_w: 51)
Coordinates: (12/17)
  * ocean_time  (ocean_time) float64 384B 4.058e+08 4.058e+08 ... 4.059e+08
  * s_w         (s_w) float64 408B -1.0 -0.98 -0.96 -0.94 ... -0.04 -0.02 0.0
  * s_rho       (s_rho) float64 400B -0.99 -0.97 -0.95 ... -0.05 -0.03 -0.01
    lon_psi     (eta_v, xi_u) float64 12MB dask.array<chunksize=(851, 1701), meta=np.ndarray>
    lat_psi     (eta_v, xi_u) float64 12MB dask.array<chunksize=(851, 1701), meta=np.ndarray>
  * xi_rho      (xi_rho) int64 14kB 0 1 2 3 4 5 ... 1697 1698 1699 1700 1701
    ...          ...
    z_w_v       (ocean_time, s_w, eta_v, xi_rho) float64 28GB dask.array<chunksize=(1, 51, 851, 1702), meta=np.ndarray>
    z_w_psi     (ocean_time, s_w, eta_v, xi_u) float64 28GB dask.array<chunksize=(1, 51, 851, 1701), meta=np.ndarray>
    z_rho       (ocean_time, s_rho, eta_rho, xi_rho) float64 28GB dask.array<chunksize=(1, 50, 852, 1702), meta=np.ndarray>
    z_rho_u     (ocean_time, s_rho, eta_rho, xi_u) float64 28GB dask.array<chunksize=(1, 50, 852, 1701), meta=np.ndarray>
    z_rho_v     (ocean_time, s_rho, eta_v, xi_rho) float64 28GB dask.array<chunksize=(1, 50, 851, 1702), meta=np.ndarray>
    z_rho_psi   (ocean_time, s_rho, eta_v, xi_u) float64 28GB dask.array<chunksize=(1, 50, 851, 1701), meta=np.ndarray>
Data variables: (12/50)
    zeta        (ocean_time, eta_rho, xi_rho) float32 278MB dask.array<chunksize=(1, 852, 1702), meta=np.ndarray>
    ubar        (ocean_time, eta_rho, xi_u) float32 278MB dask.array<chunksize=(1, 852, 1701), meta=np.ndarray>
    vbar        (ocean_time, eta_v, xi_rho) float32 278MB dask.array<chunksize=(1, 851, 1702), meta=np.ndarray>
    u           (ocean_time, s_rho, eta_rho, xi_u) float32 14GB dask.array<chunksize=(1, 50, 852, 1701), meta=np.ndarray>
    v           (ocean_time, s_rho, eta_v, xi_rho) float32 14GB dask.array<chunksize=(1, 50, 851, 1702), meta=np.ndarray>
    temp        (ocean_time, s_rho, eta_rho, xi_rho) float32 14GB dask.array<chunksize=(1, 50, 852, 1702), meta=np.ndarray>
    ...          ...
    dz_v        (ocean_time, s_rho, eta_v, xi_rho) float64 28GB dask.array<chunksize=(1, 50, 851, 1702), meta=np.ndarray>
    dz_w_v      (ocean_time, s_w, eta_v, xi_rho) float64 28GB dask.array<chunksize=(1, 51, 851, 1702), meta=np.ndarray>
    dz_psi      (ocean_time, s_rho, eta_v, xi_u) float64 28GB dask.array<chunksize=(1, 50, 851, 1701), meta=np.ndarray>
    dz_w_psi    (ocean_time, s_w, eta_v, xi_u) float64 28GB dask.array<chunksize=(1, 51, 851, 1701), meta=np.ndarray>
    dA          (eta_rho, xi_rho) float64 12MB dask.array<chunksize=(852, 1702), meta=np.ndarray>
    rho0        int64 8B 1025
Attributes: (12/33)
    type:           ROMS averages file
    title:          L3 MidCal 100 m 50 levels (no wave)
    grid_file:      usw3_grd.000.nc
    init_file:      usw3_rst.000.nc
    ntimes:         1209600
    ndtfast:        50
    ...             ...
    rdrg2_units:    nondimensional
    Zob:            0.01
    Zob_units:      meter
    SRCS:           KRNSRC main.F step2D_FB.F read_inp.F read_sta_pos.F timer...
    CPPS:           <cppdefs.h> USWC SOLVE3D UV_COR UV_ADV CURVGRID SPHERICAL...
    nc_format:      netCDF-4, zlib-compressed

In [ ]:
ds["vort"] = ds.xroms.vort
ds.vort.compute()

/home/groups/leift/hilditch/projects/roms_analysis_demo/project_code/.venv/lib/python3.12/site-packages/distributed/client.py:3370: UserWarning: Sending large graph of size 56.41 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
